<a href="https://colab.research.google.com/github/chainiqedu/chainiqedu/blob/main/volmex_finance_arbitrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Volmex Finance: Arbitrum 

Analysis of Depositors, Collateral Growth, Liquidity Providers and Traders

In [649]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [650]:
# Read in data from CSV
minters_df = pd.read_csv('/content/minters_v4_final_fixed.csv')
minters_df = minters_df.drop(['Unnamed: 0'], axis=1)
minters_df['date_formatted'] = pd.to_datetime(minters_df['DateTime']).dt.date

lps_traders_df = pd.read_csv('/content/lps_traders_v4_final.csv')
lps_traders_df = lps_traders_df.drop(['Unnamed: 0'], axis=1)
lps_traders_df['date_formatted'] = pd.to_datetime(lps_traders_df['DateTime']).dt.date

# Filter for minters/lps/traders on Arbitrum chain
arbitrum_minters_df = minters_df[minters_df['chain'] == 'ARBITRUM'].reset_index(drop=True)
arbitrum_lps_traders_df = lps_traders_df[lps_traders_df['chain'] == 'ARBITRUM'].reset_index(drop=True)

In [651]:
# Add an additional column to the dataframe to indicate which collateral pool is being deposited to 
def collateral_pool(row):
    if row['To'] == '0xe46277336d9cc2ebe7b24ba7268624f5f1495611':
        return 'BTCV-DAI'
    elif row['To']== '0xf613b55131cf8a69c5b4f62d0d5e5d2c2d9c3280':
        return 'ETHV-DAI'
    elif row['To']== '0xdf87072ac4722431861837492edf7adbfec0efa9':
        return 'BTCV-USDC'
    elif row['To']== '0xf9b04aad2612d3d664f41e9af5711953e058ff52':
        return 'ETHV-USDC'
    elif row['From'] == '0xe46277336d9cc2ebe7b24ba7268624f5f1495611':
        return 'BTCV-DAI'
    elif row['From']== '0xf613b55131cf8a69c5b4f62d0d5e5d2c2d9c3280':
        return 'ETHV-DAI'
    elif row['From']== '0xdf87072ac4722431861837492edf7adbfec0efa9':
        return 'BTCV-USDC'
    elif row['From']== '0xf9b04aad2612d3d664f41e9af5711953e058ff52':
        return 'ETHV-USDC'
    return 'UNKNOWN'

arbitrum_minters_df["collateral_pool"] = arbitrum_minters_df.apply(collateral_pool, axis=1)

## Arbitrum Depositors

In [652]:
# Depositors
arbitrum_depositors_df = arbitrum_minters_df[arbitrum_minters_df['type'] == 'COLLATERALIZE']
# Calculate number of unique depositors
arbitrum_unique_depositors = arbitrum_minters_df[arbitrum_minters_df['type'] == 'COLLATERALIZE']['From'].nunique()

In [653]:
# Create deposit date series sequence for cumulative calculations etc
min_date = arbitrum_depositors_df['date_formatted'].min()
max_date = arbitrum_depositors_df['date_formatted'].max()
arbitrum_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=min_date, end=max_date).values})
arbitrum_date_series_df['date_formatted'] = pd.to_datetime(arbitrum_date_series_df['date_formatted']).dt.date

In [654]:
# Calculate cumulative unique depositors over time
arbitrum_depositors_min_date_df = arbitrum_depositors_df.groupby('From')['date_formatted'].agg('min').reset_index()
arbitrum_depositors_min_date_df = arbitrum_depositors_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
arbitrum_depositors_min_date_series_df = arbitrum_date_series_df.merge(arbitrum_depositors_min_date_df, on='date_formatted', how='left').fillna(0)
arbitrum_depositors_min_date_series_df['cumulative_depositors'] = arbitrum_depositors_min_date_series_df['From'].cumsum(axis=0)

In [655]:
# Calculate unique depositors per day over time
arbitrum_unique_depositors_per_day_df = arbitrum_depositors_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
arbitrum_unique_depositors_series_df = arbitrum_date_series_df.merge(arbitrum_unique_depositors_per_day_df, on='date_formatted', how='left').fillna(0)

In [656]:
# Calculate deposit values per day
arbitrum_deposit_values_per_day_df = arbitrum_depositors_df.groupby('date_formatted').agg({'Value' : ['min', 'max', 'mean', 'median']})
arbitrum_deposit_values_per_day_df = arbitrum_date_series_df.merge(arbitrum_deposit_values_per_day_df, on='date_formatted', how='left').fillna(0)

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning:

merging between different levels can give an unintended result (1 levels on the left,2 on the right)



In [657]:
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "indicator"}, {"type": "bar"}],
          [{"type": "bar"}, {"type": "scatter"}]],
    subplot_titles=("Arbitrum Unique Depositors",
                    "Arbitrum Cumulative Unique Depositors", 
                    "Arbitrum Unique Depositors Per Day",
                    "Arbitrum Min/Max/Avg/Median Deposit Values Per Day")
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = arbitrum_unique_depositors),
row=1, col=1)

fig.add_trace(go.Bar(
    x = arbitrum_depositors_min_date_series_df['date_formatted'],
    y = arbitrum_depositors_min_date_series_df['cumulative_depositors'].values,
    name='cumulative unique depositors'),
    row=1, col=2)

fig.add_trace(go.Bar(
    x = arbitrum_unique_depositors_series_df['date_formatted'].values,
    y = arbitrum_unique_depositors_series_df['From'].values,
    name='unique depositors'),
    row=2, col=1)

fig.add_trace(go.Scatter(
    x = arbitrum_deposit_values_per_day_df['date_formatted'].values,
    y = arbitrum_deposit_values_per_day_df[('Value', 'min')].values,
    name='min'),
    row=2, col=2)

fig.add_trace(go.Scatter(
    x = arbitrum_deposit_values_per_day_df['date_formatted'].values,
    y = arbitrum_deposit_values_per_day_df[('Value', 'max')].values,
    name='max'),
    row=2, col=2)

fig.add_trace(go.Scatter(
    x = arbitrum_deposit_values_per_day_df['date_formatted'].values,
    y = arbitrum_deposit_values_per_day_df[('Value', 'mean')].values,
    name='mean'),
    row=2, col=2)

fig.add_trace(go.Scatter(
    x = arbitrum_deposit_values_per_day_df['date_formatted'].values,
    y = arbitrum_deposit_values_per_day_df[('Value', 'median')].values,
    name='median'),
    row=2, col=2)

# # Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)

fig.update_yaxes(title_text="Cumulative Unique Depositors", row=1, col=2)
fig.update_yaxes(title_text="Num Unique Depositors", row=2, col=1)
fig.update_yaxes(title_text="Deposit Value $", row=2, col=2)

fig.update_layout(height=800, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

## Arbitrum Depositors - DAI Collateral

In [658]:
# DAI Depositors
btcv_dai_depositors_df = arbitrum_depositors_df[(arbitrum_depositors_df['collateral_pool'] == 'BTCV-DAI') & 
                                                (arbitrum_depositors_df['type'] == 'COLLATERALIZE')]
ethv_dai_depositors_df = arbitrum_depositors_df[(arbitrum_depositors_df['collateral_pool'] == 'ETHV-DAI') &
                                                (arbitrum_depositors_df['type'] == 'COLLATERALIZE')]
btcv_usdc_depositors_df = arbitrum_depositors_df[(arbitrum_depositors_df['collateral_pool'] == 'BTCV-USDC') &
                                                 (arbitrum_depositors_df['type'] == 'COLLATERALIZE')]
ethv_usdc_depositors_df = arbitrum_depositors_df[(arbitrum_depositors_df['collateral_pool'] == 'ETHV-USDC') &
                                                 (arbitrum_depositors_df['type'] == 'COLLATERALIZE')]

# Calculate number of unique DAI depositors
btcv_dai_unique_depositors = btcv_dai_depositors_df['From'].nunique()
ethv_dai_unique_depositors = ethv_dai_depositors_df['From'].nunique()
btcv_usdc_unique_depositors = btcv_usdc_depositors_df['From'].nunique()
ethv_usdc_unique_depositors = ethv_usdc_depositors_df['From'].nunique()

In [659]:
# Create deposit date series sequence for cumulative calculations etc for BTCV-DAI & ETHV-DAI
btcv_dai_min_date = btcv_dai_depositors_df['date_formatted'].min()
btcv_dai_max_date = btcv_dai_depositors_df['date_formatted'].max()

ethv_dai_min_date = ethv_dai_depositors_df['date_formatted'].min()
ethv_dai_max_date = ethv_dai_depositors_df['date_formatted'].max()

btcv_usdc_min_date = btcv_usdc_depositors_df['date_formatted'].min()
btcv_usdc_max_date = btcv_usdc_depositors_df['date_formatted'].max()

ethv_usdc_min_date = ethv_usdc_depositors_df['date_formatted'].min()
ethv_usdc_max_date = ethv_usdc_depositors_df['date_formatted'].max()

btcv_dai_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=btcv_dai_min_date, end=btcv_dai_max_date).values})
ethv_dai_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=ethv_dai_min_date, end=ethv_dai_max_date).values})
btcv_usdc_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=btcv_usdc_min_date, end=btcv_usdc_max_date).values})
ethv_usdc_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=ethv_usdc_min_date, end=ethv_usdc_max_date).values})

btcv_dai_date_series_df['date_formatted'] = pd.to_datetime(btcv_dai_date_series_df['date_formatted']).dt.date
ethv_dai_date_series_df['date_formatted'] = pd.to_datetime(ethv_dai_date_series_df['date_formatted']).dt.date
btcv_usdc_date_series_df['date_formatted'] = pd.to_datetime(btcv_usdc_date_series_df['date_formatted']).dt.date
ethv_usdc_date_series_df['date_formatted'] = pd.to_datetime(ethv_usdc_date_series_df['date_formatted']).dt.date

In [660]:
# Calculate cumulative unique depositors over time
btcv_dai_depositors_min_date_df = btcv_dai_depositors_df.groupby('From')['date_formatted'].agg('min').reset_index()
btcv_dai_depositors_min_date_df = btcv_dai_depositors_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
btcv_dai_depositors_min_date_series_df = btcv_dai_date_series_df.merge(btcv_dai_depositors_min_date_df, on='date_formatted', how='left').fillna(0)
btcv_dai_depositors_min_date_series_df['cumulative_depositors'] = btcv_dai_depositors_min_date_series_df['From'].cumsum(axis=0)

ethv_dai_depositors_min_date_df = ethv_dai_depositors_df.groupby('From')['date_formatted'].agg('min').reset_index()
ethv_dai_depositors_min_date_df = ethv_dai_depositors_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
ethv_dai_depositors_min_date_series_df = ethv_dai_date_series_df.merge(ethv_dai_depositors_min_date_df, on='date_formatted', how='left').fillna(0)
ethv_dai_depositors_min_date_series_df['cumulative_depositors'] = ethv_dai_depositors_min_date_series_df['From'].cumsum(axis=0)

btcv_usdc_depositors_min_date_df = btcv_usdc_depositors_df.groupby('From')['date_formatted'].agg('min').reset_index()
btcv_usdc_depositors_min_date_df = btcv_usdc_depositors_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
btcv_usdc_depositors_min_date_series_df = btcv_usdc_date_series_df.merge(btcv_usdc_depositors_min_date_df, on='date_formatted', how='left').fillna(0)
btcv_usdc_depositors_min_date_series_df['cumulative_depositors'] = btcv_usdc_depositors_min_date_series_df['From'].cumsum(axis=0)

ethv_usdc_depositors_min_date_df = ethv_usdc_depositors_df.groupby('From')['date_formatted'].agg('min').reset_index()
ethv_usdc_depositors_min_date_df = ethv_usdc_depositors_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
ethv_usdc_depositors_min_date_series_df = ethv_usdc_date_series_df.merge(ethv_usdc_depositors_min_date_df, on='date_formatted', how='left').fillna(0)
ethv_usdc_depositors_min_date_series_df['cumulative_depositors'] = ethv_usdc_depositors_min_date_series_df['From'].cumsum(axis=0)

In [661]:
# Calculate unique depositors per day over time
btcv_dai_unique_depositors_per_day_df = btcv_dai_depositors_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
btcv_dai_unique_depositors_series_df = btcv_dai_date_series_df.merge(btcv_dai_unique_depositors_per_day_df, on='date_formatted', how='left').fillna(0)

ethv_dai_unique_depositors_per_day_df = ethv_dai_depositors_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
ethv_dai_unique_depositors_series_df = ethv_dai_date_series_df.merge(ethv_dai_unique_depositors_per_day_df, on='date_formatted', how='left').fillna(0)

btcv_usdc_unique_depositors_per_day_df = btcv_usdc_depositors_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
btcv_usdc_unique_depositors_series_df = btcv_usdc_date_series_df.merge(btcv_usdc_unique_depositors_per_day_df, on='date_formatted', how='left').fillna(0)

ethv_usdc_unique_depositors_per_day_df = ethv_usdc_depositors_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
ethv_usdc_unique_depositors_series_df = ethv_usdc_date_series_df.merge(ethv_usdc_unique_depositors_per_day_df, on='date_formatted', how='left').fillna(0)

In [662]:
# Calculate deposit values per day
btcv_dai_deposit_values_per_day_df = btcv_dai_depositors_df.groupby('date_formatted').agg({'Value' : ['min', 'max', 'mean', 'median']})
btcv_dai_deposit_values_per_day_df = btcv_dai_date_series_df.merge(btcv_dai_deposit_values_per_day_df, on='date_formatted', how='left').fillna(0)

ethv_dai_deposit_values_per_day_df = ethv_dai_depositors_df.groupby('date_formatted').agg({'Value' : ['min', 'max', 'mean', 'median']})
ethv_dai_deposit_values_per_day_df = ethv_dai_date_series_df.merge(ethv_dai_deposit_values_per_day_df, on='date_formatted', how='left').fillna(0)

btcv_usdc_deposit_values_per_day_df = btcv_usdc_depositors_df.groupby('date_formatted').agg({'Value' : ['min', 'max', 'mean', 'median']})
btcv_usdc_deposit_values_per_day_df = btcv_usdc_date_series_df.merge(btcv_usdc_deposit_values_per_day_df, on='date_formatted', how='left').fillna(0)

ethv_usdc_deposit_values_per_day_df = ethv_usdc_depositors_df.groupby('date_formatted').agg({'Value' : ['min', 'max', 'mean', 'median']})
ethv_usdc_deposit_values_per_day_df = ethv_usdc_date_series_df.merge(ethv_usdc_deposit_values_per_day_df, on='date_formatted', how='left').fillna(0)

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning:

merging between different levels can give an unintended result (1 levels on the left,2 on the right)



In [663]:
fig = make_subplots(
    rows=4, cols=2,
    specs=[[{"type": "indicator"}, {"type": "indicator"}],
           [{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}],
          [{"type": "scatter"}, {"type": "scatter"}]],
    subplot_titles=("BTCV-DAI Unique Depositors",
                    "ETHV-DAI Unique Depositors",
                    "BTCV-DAI Cumulative Unique Depositors",
                    "ETHV-DAI Cumulative Unique Depositors",
                    "BTCV-DAI Unique Depositors Per Day",
                    "ETHV-DAI Unique Depositors Per Day",
                    "BTCV-DAI Min/Max/Avg/Median Deposit Values Per Day",
                    "ETHV-DAI Min/Max/Avg/Median Deposit Values Per Day")
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = btcv_dai_unique_depositors),
row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    value = ethv_dai_unique_depositors),
row=1, col=2)

fig.add_trace(go.Bar(
    x = btcv_dai_depositors_min_date_series_df['date_formatted'],
    y = btcv_dai_depositors_min_date_series_df['cumulative_depositors'].values,
    name='btcv_dai_cumulative_unique_depositors'),
    row=2, col=1)

fig.add_trace(go.Bar(
    x = ethv_dai_depositors_min_date_series_df['date_formatted'],
    y = ethv_dai_depositors_min_date_series_df['cumulative_depositors'].values,
    name='ethv_dai_cumulative_unique_depositors'),
    row=2, col=2)

fig.add_trace(go.Bar(
    x = btcv_dai_unique_depositors_series_df['date_formatted'].values,
    y = btcv_dai_unique_depositors_series_df['From'].values,
    name='btcv_dai_unique_depositors'),
    row=3, col=1)

fig.add_trace(go.Bar(
    x = ethv_dai_unique_depositors_series_df['date_formatted'].values,
    y = ethv_dai_unique_depositors_series_df['From'].values,
    name='ethv_dai_unique_depositors'),
    row=3, col=2)

fig.add_trace(go.Scatter(
    x = btcv_dai_deposit_values_per_day_df['date_formatted'].values,
    y = btcv_dai_deposit_values_per_day_df[('Value', 'min')].values,
    name='btcv_dai_deposit_min'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = btcv_dai_deposit_values_per_day_df['date_formatted'].values,
    y = btcv_dai_deposit_values_per_day_df[('Value', 'max')].values,
    name='btcv_dai_max'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = btcv_dai_deposit_values_per_day_df['date_formatted'].values,
    y = btcv_dai_deposit_values_per_day_df[('Value', 'mean')].values,
    name='btcv_dai_deposit_mean'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = btcv_dai_deposit_values_per_day_df['date_formatted'].values,
    y = btcv_dai_deposit_values_per_day_df[('Value', 'median')].values,
    name='btcv_dai_deposit_median'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = ethv_dai_deposit_values_per_day_df['date_formatted'].values,
    y = ethv_dai_deposit_values_per_day_df[('Value', 'min')].values,
    name='ethv_dai_deposit_min'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = ethv_dai_deposit_values_per_day_df['date_formatted'].values,
    y = ethv_dai_deposit_values_per_day_df[('Value', 'max')].values,
    name='ethv_dai_deposit_max'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = ethv_dai_deposit_values_per_day_df['date_formatted'].values,
    y = ethv_dai_deposit_values_per_day_df[('Value', 'mean')].values,
    name='ethv_dai_deposit_mean'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = ethv_dai_deposit_values_per_day_df['date_formatted'].values,
    y = ethv_dai_deposit_values_per_day_df[('Value', 'median')].values,
    name='ethv_dai_deposit_median'),
    row=4, col=2)

# # Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=2)
fig.update_xaxes(title_text="Date", row=4, col=1)
fig.update_xaxes(title_text="Date", row=4, col=2)


fig.update_yaxes(title_text="Cumulative Unique Depositors", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Unique Depositors", row=2, col=2)

fig.update_yaxes(title_text="Num Unique Depositors", row=3, col=1)
fig.update_yaxes(title_text="Num Unique Depositors", row=3, col=2)

fig.update_yaxes(title_text="Deposit Value $", row=4, col=1)
fig.update_yaxes(title_text="Deposit Value $", row=4, col=2)

fig.update_layout(height=1000, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

## Arbitrum Depositors - USDC Collateral

In [664]:
fig = make_subplots(
    rows=4, cols=2,
    specs=[[{"type": "indicator"}, {"type": "indicator"}],
           [{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}],
          [{"type": "scatter"}, {"type": "scatter"}]],
    subplot_titles=("BTCV-USDC Unique Depositors",
                    "ETHV-USDC Unique Depositors",
                    "BTCV-USDC Cumulative Unique Depositors",
                    "ETHV-USDC Cumulative Unique Depositors",
                    "BTCV-USDC Unique Depositors Per Day",
                    "ETHV-USDC Unique Depositors Per Day",
                    "BTCV-USDC Min/Max/Avg/Median Deposit Values Per Day",
                    "ETHV-USDC Min/Max/Avg/Median Deposit Values Per Day")
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = btcv_usdc_unique_depositors),
row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    value = ethv_usdc_unique_depositors),
row=1, col=2)

fig.add_trace(go.Bar(
    x = btcv_usdc_depositors_min_date_series_df['date_formatted'],
    y = btcv_usdc_depositors_min_date_series_df['cumulative_depositors'].values,
    name='btcv_usdc_cumulative_unique_depositors'),
    row=2, col=1)

fig.add_trace(go.Bar(
    x = ethv_usdc_depositors_min_date_series_df['date_formatted'],
    y = ethv_usdc_depositors_min_date_series_df['cumulative_depositors'].values,
    name='ethv_usdc_cumulative_unique_depositors'),
    row=2, col=2)

fig.add_trace(go.Bar(
    x = btcv_usdc_unique_depositors_series_df['date_formatted'].values,
    y = btcv_usdc_unique_depositors_series_df['From'].values,
    name='btcv_usdc_unique_depositors'),
    row=3, col=1)

fig.add_trace(go.Bar(
    x = ethv_usdc_unique_depositors_series_df['date_formatted'].values,
    y = ethv_usdc_unique_depositors_series_df['From'].values,
    name='ethv_usdc_unique_depositors'),
    row=3, col=2)

fig.add_trace(go.Scatter(
    x = btcv_usdc_deposit_values_per_day_df['date_formatted'].values,
    y = btcv_usdc_deposit_values_per_day_df[('Value', 'min')].values,
    name='btcv_usdc_deposit_min'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = btcv_usdc_deposit_values_per_day_df['date_formatted'].values,
    y = btcv_usdc_deposit_values_per_day_df[('Value', 'max')].values,
    name='btcv_usdc_max'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = btcv_usdc_deposit_values_per_day_df['date_formatted'].values,
    y = btcv_usdc_deposit_values_per_day_df[('Value', 'mean')].values,
    name='btcv_usdc_deposit_mean'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = btcv_usdc_deposit_values_per_day_df['date_formatted'].values,
    y = btcv_usdc_deposit_values_per_day_df[('Value', 'median')].values,
    name='btcv_usdc_deposit_median'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = ethv_usdc_deposit_values_per_day_df['date_formatted'].values,
    y = ethv_usdc_deposit_values_per_day_df[('Value', 'min')].values,
    name='ethv_usdc_deposit_min'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = ethv_usdc_deposit_values_per_day_df['date_formatted'].values,
    y = ethv_usdc_deposit_values_per_day_df[('Value', 'max')].values,
    name='ethv_usdc_deposit_max'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = ethv_usdc_deposit_values_per_day_df['date_formatted'].values,
    y = ethv_usdc_deposit_values_per_day_df[('Value', 'mean')].values,
    name='ethv_usdc_deposit_mean'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = ethv_dai_deposit_values_per_day_df['date_formatted'].values,
    y = ethv_dai_deposit_values_per_day_df[('Value', 'median')].values,
    name='ethv_usdc_deposit_median'),
    row=4, col=2)

# # Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=2)
fig.update_xaxes(title_text="Date", row=4, col=1)
fig.update_xaxes(title_text="Date", row=4, col=2)


fig.update_yaxes(title_text="Cumulative Unique Depositors", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Unique Depositors", row=2, col=2)

fig.update_yaxes(title_text="Num Unique Depositors", row=3, col=1)
fig.update_yaxes(title_text="Num Unique Depositors", row=3, col=2)

fig.update_yaxes(title_text="Deposit Value $", row=4, col=1)
fig.update_yaxes(title_text="Deposit Value $", row=4, col=2)

fig.update_layout(height=1000, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

## Arbitrum Collateral TVL & Growth

In [665]:
def redeem_negative_value(row):
    if row['type'] == 'REDEEM/OTHER':
        return - row['Value']
    return row['Value']

In [666]:
arbitrum_deposit_redeem_df = arbitrum_minters_df
arbitrum_deposit_redeem_df['Value'] = arbitrum_deposit_redeem_df.apply(redeem_negative_value, axis=1)

# DAI Collateral Pools
arbitrum_btcv_dai_deposit_redeem_df = arbitrum_minters_df[(arbitrum_minters_df['TokenSymbol'] == 'DAI') &
                                                          (arbitrum_minters_df['collateral_pool'] == 'BTCV-DAI')]

arbitrum_ethv_dai_deposit_redeem_df = arbitrum_minters_df[(arbitrum_minters_df['TokenSymbol'] == 'DAI') &
                                                                   (arbitrum_minters_df['collateral_pool'] == 'ETHV-DAI')]

# USDC Collateral Pools

arbitrum_btcv_usdc_deposit_redeem_df = arbitrum_minters_df[(arbitrum_minters_df['TokenSymbol'] == 'USDC') &
                                                          (arbitrum_minters_df['collateral_pool'] == 'BTCV-USDC')]

arbitrum_ethv_usdc_deposit_redeem_df = arbitrum_minters_df[(arbitrum_minters_df['TokenSymbol'] == 'USDC') &
                                                           (arbitrum_minters_df['collateral_pool'] == 'ETHV-USDC')]                       

In [667]:
# Create the date series for all tx's in arbitrum collateral pools
arbitrum_minters_min_date = arbitrum_minters_df['date_formatted'].min()
arbitrum_minters_max_date = arbitrum_minters_df['date_formatted'].max()

arbitrum_minters_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=arbitrum_minters_min_date, end=arbitrum_minters_max_date).values})
arbitrum_minters_date_series_df['date_formatted'] = pd.to_datetime(arbitrum_minters_date_series_df['date_formatted']).dt.date

In [668]:
# Arbitrum Cumulative Collateral TVL
arbitrum_deposit_redeem_date_series_df = arbitrum_minters_date_series_df.merge(arbitrum_deposit_redeem_df, on='date_formatted', how='left').fillna(0)
arbitrum_net_deposit_redeem_date_series_df = arbitrum_deposit_redeem_date_series_df.groupby('date_formatted').agg({'Value': 'sum'}).reset_index()
arbitrum_net_deposit_redeem_date_series_df['cumulative_collateral_tvl'] = arbitrum_net_deposit_redeem_date_series_df['Value'].cumsum(axis=0)
arbitrum_current_tvl = arbitrum_net_deposit_redeem_date_series_df.iloc[-1]['cumulative_collateral_tvl']

In [669]:
# DAI Cumulative Collateral TVL
arbitrum_btcv_dai_deposit_redeem_date_series_df = arbitrum_minters_date_series_df.merge(arbitrum_btcv_dai_deposit_redeem_df, on='date_formatted', how='left').fillna(0)
arbitrum_btcv_dai_net_deposit_redeem_date_series_df = arbitrum_btcv_dai_deposit_redeem_date_series_df.groupby('date_formatted').agg({'Value': 'sum'}).reset_index()
arbitrum_btcv_dai_net_deposit_redeem_date_series_df['cumulative_collateral_tvl'] = arbitrum_btcv_dai_net_deposit_redeem_date_series_df['Value'].cumsum(axis=0)
arbitrum_btcv_dai_current_tvl = arbitrum_btcv_dai_net_deposit_redeem_date_series_df.iloc[-1]['cumulative_collateral_tvl']

arbitrum_ethv_dai_deposit_redeem_date_series_df = arbitrum_minters_date_series_df.merge(arbitrum_ethv_dai_deposit_redeem_df, on='date_formatted', how='left').fillna(0)
arbitrum_ethv_dai_net_deposit_redeem_date_series_df = arbitrum_ethv_dai_deposit_redeem_date_series_df.groupby('date_formatted').agg({'Value': 'sum'}).reset_index()
arbitrum_ethv_dai_net_deposit_redeem_date_series_df['cumulative_collateral_tvl'] = arbitrum_ethv_dai_net_deposit_redeem_date_series_df['Value'].cumsum(axis=0)
arbitrum_ethv_dai_current_tvl = arbitrum_ethv_dai_net_deposit_redeem_date_series_df.iloc[-1]['cumulative_collateral_tvl']

In [670]:
# USDC Cumulative Collateral TVL
arbitrum_btcv_usdc_deposit_redeem_date_series_df = arbitrum_minters_date_series_df.merge(arbitrum_btcv_usdc_deposit_redeem_df, on='date_formatted', how='left').fillna(0)
arbitrum_btcv_usdc_net_deposit_redeem_date_series_df = arbitrum_btcv_usdc_deposit_redeem_date_series_df.groupby('date_formatted').agg({'Value': 'sum'}).reset_index()
arbitrum_btcv_usdc_net_deposit_redeem_date_series_df['cumulative_collateral_tvl'] = arbitrum_btcv_usdc_net_deposit_redeem_date_series_df['Value'].cumsum(axis=0)
arbitrum_btcv_usdc_current_tvl = arbitrum_btcv_usdc_net_deposit_redeem_date_series_df.iloc[-1]['cumulative_collateral_tvl']

arbitrum_ethv_usdc_deposit_redeem_date_series_df = arbitrum_minters_date_series_df.merge(arbitrum_ethv_usdc_deposit_redeem_df, on='date_formatted', how='left').fillna(0)
arbitrum_ethv_usdc_net_deposit_redeem_date_series_df = arbitrum_ethv_usdc_deposit_redeem_date_series_df.groupby('date_formatted').agg({'Value': 'sum'}).reset_index()
arbitrum_ethv_usdc_net_deposit_redeem_date_series_df['cumulative_collateral_tvl'] = arbitrum_ethv_usdc_net_deposit_redeem_date_series_df['Value'].cumsum(axis=0)
arbitrum_ethv_usdc_current_tvl = arbitrum_ethv_usdc_net_deposit_redeem_date_series_df.iloc[-1]['cumulative_collateral_tvl']

In [671]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "indicator"}, {"type": "scatter"}]],
    subplot_titles=("Arbitrum Current TVL",
                    "Arbitrum Collateral Growth")
)

fig.add_trace(go.Indicator(
    mode = "number",
    number = {'prefix': "$"},
    value = arbitrum_current_tvl),
row=1, col=1)

fig.add_trace(go.Scatter(
    x = arbitrum_net_deposit_redeem_date_series_df['date_formatted'],
    y = arbitrum_net_deposit_redeem_date_series_df['cumulative_collateral_tvl'].values,
    name='Arbitrum Collateral Growth'),
    row=1, col=2)


fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_yaxes(title_text="Collateral Locked $", row=1, col=2)

fig.update_layout(height=500, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

## Arbitrum DAI Collateral Growth & TVL

In [672]:
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "indicator"}, {"type": "indicator"}],
           [{"type": "scatter"}, {"type": "scatter"}]],
    subplot_titles=("Arbitrum BTCV-DAI Current TVL",
                    "Arbitrum ETHV-DAI Current TVL",
                    "Arbitrum BTCV-DAI Collateral Growth",
                    "Arbitrum ETHV-DAI Collateral Growth")
)

fig.add_trace(go.Indicator(
    mode = "number",
    number = {'prefix': "$"},
    value = arbitrum_btcv_dai_current_tvl),
row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    number = {'prefix': "$"},
    value = arbitrum_ethv_dai_current_tvl),
row=1, col=2)

fig.add_trace(go.Scatter(
    x = arbitrum_btcv_dai_net_deposit_redeem_date_series_df['date_formatted'],
    y = arbitrum_btcv_dai_net_deposit_redeem_date_series_df['cumulative_collateral_tvl'].values,
    name='Arbitrum BTCV-DAI Collateral Growth'),
    row=2, col=1)

fig.add_trace(go.Scatter(
    x = arbitrum_ethv_dai_net_deposit_redeem_date_series_df['date_formatted'],
    y = arbitrum_ethv_dai_net_deposit_redeem_date_series_df['cumulative_collateral_tvl'].values,
    name='Arbitrum ETHV-DAI Collateral Growth'),
    row=2, col=2)


fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)

fig.update_yaxes(title_text="Collateral Locked $", row=2, col=1)
fig.update_yaxes(title_text="Collateral Locked $", row=2, col=2)

fig.update_layout(height=800, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

## Arbitrum USDC Collateral Growth & TVL

In [673]:
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "indicator"}, {"type": "indicator"}],
           [{"type": "scatter"}, {"type": "scatter"}]],
    subplot_titles=("Arbitrum BTCV-USDC Current TVL",
                    "Arbitrum ETHV-USDC Current TVL",
                    "Arbitrum BTCV-USDC Collateral Growth",
                    "Arbitrum ETHV-USDC Collateral Growth")
)

fig.add_trace(go.Indicator(
    mode = "number",
    number = {'prefix': "$"},
    value = arbitrum_btcv_usdc_current_tvl),
row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    number = {'prefix': "$"},
    value = arbitrum_ethv_usdc_current_tvl),
row=1, col=2)

fig.add_trace(go.Scatter(
    x = arbitrum_btcv_usdc_net_deposit_redeem_date_series_df['date_formatted'],
    y = arbitrum_btcv_usdc_net_deposit_redeem_date_series_df['cumulative_collateral_tvl'].values,
    name='Arbitrum BTCV-USDC Collateral Growth'),
    row=2, col=1)

fig.add_trace(go.Scatter(
    x = arbitrum_ethv_usdc_net_deposit_redeem_date_series_df['date_formatted'],
    y = arbitrum_ethv_usdc_net_deposit_redeem_date_series_df['cumulative_collateral_tvl'].values,
    name='Arbitrum ETHV-USDC Collateral Growth'),
    row=2, col=2)


fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)

fig.update_yaxes(title_text="Collateral Locked $", row=2, col=1)
fig.update_yaxes(title_text="Collateral Locked $", row=2, col=2)

fig.update_layout(height=800, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

## Arbitrum Volatility Token Liquidity Providers

In [674]:
# arbitrum_lps_traders_df['tx_type_label'].unique()
arbitrum_lps_df = arbitrum_lps_traders_df[arbitrum_lps_traders_df['tx_type_label'] == 'Add Liquidity']

In [675]:
def exchange_pool(row):
    if row['To'] == '0x3396e2cb9e4f1eb4fc06028ab07a8db0ef63b486':
        return 'BTCV-USDC'
    elif row['To']== '0x8278591bbbbe7d86e783507e7f6320ffc9c021e5':
        return 'ETHV-USDC'
    elif row['To']== '0xc80956377cea73f64aaa3613d6692eabdf4fd4c8':
        return 'iBTCV-USDC'
    elif row['To']== '0x5497ca4be41d2ae0222cc4c41c43d189ed894ffd':
        return 'iETHV-USDC'
    return 'UNKNOWN'

arbitrum_lps_df["exchange_pool"] = arbitrum_lps_df.apply(exchange_pool, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [676]:
# Arbitrum LPs
btcv_usdc_lps_df = arbitrum_lps_df[(arbitrum_lps_df['exchange_pool'] == 'BTCV-USDC')]
ethv_usdc_lps_df = arbitrum_lps_df[(arbitrum_lps_df['exchange_pool'] == 'ETHV-USDC')]
ibtcv_usdc_lps_df = arbitrum_lps_df[(arbitrum_lps_df['exchange_pool'] == 'iBTCV-USDC')]
iethv_usdc_lps_df = arbitrum_lps_df[(arbitrum_lps_df['exchange_pool'] == 'iETHV-USDC')]

# Calculate number of unique arbitrum LPs
arbitrum_unique_lps = arbitrum_lps_df['From'].nunique()
btcv_usdc_unique_lps = btcv_usdc_lps_df['From'].nunique()
ethv_usdc_unique_lps = ethv_usdc_lps_df['From'].nunique()
ibtcv_usdc_unique_lps = ibtcv_usdc_lps_df['From'].nunique()
iethv_usdc_unique_lps = iethv_usdc_lps_df['From'].nunique()

In [677]:
# Create LP date series sequence for cumulative calculations etc
min_date = arbitrum_lps_df['date_formatted'].min()
max_date = arbitrum_lps_df['date_formatted'].max()
arbitrum_lps_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=min_date, end=max_date).values})
arbitrum_lps_date_series_df['date_formatted'] = pd.to_datetime(arbitrum_lps_date_series_df['date_formatted']).dt.date

btcv_usdc_min_date = btcv_usdc_lps_df['date_formatted'].min()
btcv_usdc_max_date = btcv_usdc_lps_df['date_formatted'].max()

ethv_usdc_min_date = ethv_usdc_lps_df['date_formatted'].min()
ethv_usdc_max_date = ethv_usdc_lps_df['date_formatted'].max()

ibtcv_usdc_min_date = ibtcv_usdc_lps_df['date_formatted'].min()
ibtcv_usdc_max_date = ibtcv_usdc_lps_df['date_formatted'].max()

iethv_usdc_min_date = iethv_usdc_lps_df['date_formatted'].min()
iethv_usdc_max_date = iethv_usdc_lps_df['date_formatted'].max()

btcv_usdc_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=btcv_usdc_min_date, end=btcv_usdc_max_date).values})
ethv_usdc_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=ethv_usdc_min_date, end=ethv_usdc_max_date).values})
ibtcv_usdc_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=ibtcv_usdc_min_date, end=ibtcv_usdc_max_date).values})
iethv_usdc_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=iethv_usdc_min_date, end=iethv_usdc_max_date).values})

btcv_usdc_date_series_df['date_formatted'] = pd.to_datetime(btcv_usdc_date_series_df['date_formatted']).dt.date
ethv_usdc_date_series_df['date_formatted'] = pd.to_datetime(ethv_usdc_date_series_df['date_formatted']).dt.date
ibtcv_usdc_date_series_df['date_formatted'] = pd.to_datetime(ibtcv_usdc_date_series_df['date_formatted']).dt.date
iethv_usdc_date_series_df['date_formatted'] = pd.to_datetime(iethv_usdc_date_series_df['date_formatted']).dt.date

In [678]:
# Calculate cumulative LPs over time
arbitrum_lps_min_date_df = arbitrum_lps_df.groupby('From')['date_formatted'].agg('min').reset_index()
arbitrum_lps_min_date_df = arbitrum_lps_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
arbitrum_lps_min_date_series_df = arbitrum_lps_date_series_df.merge(arbitrum_lps_min_date_df, on='date_formatted', how='left').fillna(0)
arbitrum_lps_min_date_series_df['cumulative_lps'] = arbitrum_lps_min_date_series_df['From'].cumsum(axis=0)

btcv_usdc_lps_min_date_df = btcv_usdc_lps_df.groupby('From')['date_formatted'].agg('min').reset_index()
btcv_usdc_lps_min_date_df = btcv_usdc_lps_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
btcv_usdc_lps_min_date_series_df = arbitrum_lps_date_series_df.merge(btcv_usdc_lps_min_date_df, on='date_formatted', how='left').fillna(0)
btcv_usdc_lps_min_date_series_df['cumulative_lps'] = btcv_usdc_lps_min_date_series_df['From'].cumsum(axis=0)

ethv_usdc_lps_min_date_df = ethv_usdc_lps_df.groupby('From')['date_formatted'].agg('min').reset_index()
ethv_usdc_lps_min_date_df = ethv_usdc_lps_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
ethv_usdc_lps_min_date_series_df = arbitrum_lps_date_series_df.merge(ethv_usdc_lps_min_date_df, on='date_formatted', how='left').fillna(0)
ethv_usdc_lps_min_date_series_df['cumulative_lps'] = ethv_usdc_lps_min_date_series_df['From'].cumsum(axis=0)

ibtcv_usdc_lps_min_date_df = ibtcv_usdc_lps_df.groupby('From')['date_formatted'].agg('min').reset_index()
ibtcv_usdc_lps_min_date_df = ibtcv_usdc_lps_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
ibtcv_usdc_lps_min_date_series_df = arbitrum_lps_date_series_df.merge(ibtcv_usdc_lps_min_date_df, on='date_formatted', how='left').fillna(0)
ibtcv_usdc_lps_min_date_series_df['cumulative_lps'] = ibtcv_usdc_lps_min_date_series_df['From'].cumsum(axis=0)

iethv_usdc_lps_min_date_df = iethv_usdc_lps_df.groupby('From')['date_formatted'].agg('min').reset_index()
iethv_usdc_lps_min_date_df = iethv_usdc_lps_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
iethv_usdc_lps_min_date_series_df = arbitrum_lps_date_series_df.merge(iethv_usdc_lps_min_date_df, on='date_formatted', how='left').fillna(0)
iethv_usdc_lps_min_date_series_df['cumulative_lps'] = iethv_usdc_lps_min_date_series_df['From'].cumsum(axis=0)


In [679]:
# Calculate unique lps per day over time
arbitrum_unique_lps_per_day_df = arbitrum_lps_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
arbitrum_unique_lps_series_df = arbitrum_lps_date_series_df.merge(arbitrum_unique_lps_per_day_df, on='date_formatted', how='left').fillna(0)

btcv_usdc_unique_lps_per_day_df = btcv_usdc_lps_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
btcv_usdc_unique_lps_series_df = arbitrum_lps_date_series_df.merge(btcv_usdc_unique_lps_per_day_df, on='date_formatted', how='left').fillna(0)

ethv_usdc_unique_lps_per_day_df = ethv_usdc_lps_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
ethv_usdc_unique_lps_series_df = arbitrum_lps_date_series_df.merge(ethv_usdc_unique_lps_per_day_df, on='date_formatted', how='left').fillna(0)

ibtcv_usdc_unique_lps_per_day_df = ibtcv_usdc_lps_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
ibtcv_usdc_unique_lps_series_df = arbitrum_lps_date_series_df.merge(ibtcv_usdc_unique_lps_per_day_df, on='date_formatted', how='left').fillna(0)

iethv_usdc_unique_lps_per_day_df = iethv_usdc_lps_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
iethv_usdc_unique_lps_series_df = arbitrum_lps_date_series_df.merge(iethv_usdc_unique_lps_per_day_df, on='date_formatted', how='left').fillna(0)

In [680]:
# Calculate the USD value for the volatility tokens based on the latest swap from the Uniswap pool on Arbitrum
arbitrum_swaps_df = arbitrum_lps_traders_df[arbitrum_lps_traders_df['tx_type_label']== 'Swap']

latest_btcv_usdc_swap_tx_hash = arbitrum_swaps_df[arbitrum_swaps_df['TokenSymbol'] == 'BTCV'].sort_values(by=['DateTime'], ascending=False).head(1)['Txhash'].values[0]
latest_ethv_usdc_swap_tx_hash = arbitrum_swaps_df[arbitrum_swaps_df['TokenSymbol'] == 'ETHV'].sort_values(by=['DateTime'], ascending=False).head(1)['Txhash'].values[0]
latest_ibtcv_usdc_swap_tx_hash = arbitrum_swaps_df[arbitrum_swaps_df['TokenSymbol'] == 'iBTCV'].sort_values(by=['DateTime'], ascending=False).head(1)['Txhash'].values[0]
latest_iethv_usdc_swap_tx_hash = arbitrum_swaps_df[arbitrum_swaps_df['TokenSymbol'] == 'iETHV'].sort_values(by=['DateTime'], ascending=False).head(1)['Txhash'].values[0]

latest_btcv_usdc_swap_df = arbitrum_swaps_df[arbitrum_swaps_df['Txhash'] == latest_btcv_usdc_swap_tx_hash]
latest_ethv_usdc_swap_df = arbitrum_swaps_df[arbitrum_swaps_df['Txhash'] == latest_ethv_usdc_swap_tx_hash]
latest_ibtcv_usdc_swap_df = arbitrum_swaps_df[arbitrum_swaps_df['Txhash'] == latest_ibtcv_usdc_swap_tx_hash]
latest_iethv_usdc_swap_df = arbitrum_swaps_df[arbitrum_swaps_df['Txhash'] == latest_iethv_usdc_swap_tx_hash]

btcv_usd_value = latest_btcv_usdc_swap_df[latest_btcv_usdc_swap_df['TokenSymbol']=='USDC']['Value'].values[0]/latest_btcv_usdc_swap_df[latest_btcv_usdc_swap_df['TokenSymbol']=='BTCV']['Value'].values[0]
ethv_usd_value = latest_ethv_usdc_swap_df[latest_ethv_usdc_swap_df['TokenSymbol']=='USDC']['Value'].values[0]/latest_ethv_usdc_swap_df[latest_ethv_usdc_swap_df['TokenSymbol']=='ETHV']['Value'].values[0]
ibtcv_usd_value = latest_ibtcv_usdc_swap_df[latest_ibtcv_usdc_swap_df['TokenSymbol']=='USDC']['Value'].values[0]/latest_ibtcv_usdc_swap_df[latest_ibtcv_usdc_swap_df['TokenSymbol']=='iBTCV']['Value'].values[0]
iethv_usd_value = latest_iethv_usdc_swap_df[latest_iethv_usdc_swap_df['TokenSymbol']=='USDC']['Value'].values[0]/latest_iethv_usdc_swap_df[latest_iethv_usdc_swap_df['TokenSymbol']=='iETHV']['Value'].values[0]

def volatility_token_to_usd(row):
    if row['TokenSymbol'] == 'BTCV':
        return row['Value'] * btcv_usd_value
    elif row['TokenSymbol']== 'ETHV':
        return row['Value'] * ethv_usd_value
    elif row['TokenSymbol']== 'iBTCV':
        return row['Value'] * ibtcv_usd_value
    elif row['TokenSymbol']== 'iETHV':
        return row['Value'] * iethv_usd_value
    return row['Value']

arbitrum_lps_df['usd_value'] = arbitrum_lps_df.apply(volatility_token_to_usd, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [681]:
# Calculate lps values per day
arbitrum_lp_values_per_day_df = arbitrum_lps_df.groupby('date_formatted').agg({'usd_value' : ['min', 'max', 'mean', 'median']})
arbitrum_lp_values_per_day_df = arbitrum_lps_date_series_df.merge(arbitrum_lp_values_per_day_df, on='date_formatted', how='left').fillna(0)

btcv_usdc_lp_values_per_day_df = btcv_usdc_lps_df.groupby('date_formatted').agg({'Value' : ['min', 'max', 'mean', 'median']}).reset_index()
btcv_usdc_lp_values_per_day_df = arbitrum_lps_date_series_df.merge(btcv_usdc_lp_values_per_day_df, on='date_formatted', how='left').fillna(0)

ethv_usdc_lp_values_per_day_df = ethv_usdc_lps_df.groupby('date_formatted').agg({'Value' : ['min', 'max', 'mean', 'median']}).reset_index()
ethv_usdc_lp_values_per_day_df = arbitrum_lps_date_series_df.merge(ethv_usdc_lp_values_per_day_df, on='date_formatted', how='left').fillna(0)

ibtcv_usdc_lp_values_per_day_df = ibtcv_usdc_lps_df.groupby('date_formatted').agg({'Value' : ['min', 'max', 'mean', 'median']})
ibtcv_usdc_lp_values_per_day_df = arbitrum_lps_date_series_df.merge(ibtcv_usdc_lp_values_per_day_df, on='date_formatted', how='left').fillna(0)

iethv_usdc_lp_values_per_day_df = iethv_usdc_lps_df.groupby('date_formatted').agg({'Value' : ['min', 'max', 'mean', 'median']})
iethv_usdc_lp_values_per_day_df = arbitrum_lps_date_series_df.merge(iethv_usdc_lp_values_per_day_df, on='date_formatted', how='left').fillna(0)

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning:

merging between different levels can give an unintended result (1 levels on the left,2 on the right)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:3889: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [682]:
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "indicator"}, {"type": "bar"}],
          [{"type": "bar"}, {"type": "scatter"}]],
    subplot_titles=("Arbitrum Unique LPs",
                    "Arbitrum Cumulative Unique LPs", 
                    "Arbitrum Unique LPs Per Day",
                    "Arbitrum Min/Max/Avg/Median LP Values Per Day")
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = arbitrum_unique_lps),
row=1, col=1)

fig.add_trace(go.Bar(
    x = arbitrum_lps_min_date_series_df['date_formatted'],
    y = arbitrum_lps_min_date_series_df['cumulative_lps'].values,
    name='cumulative unique LPs'),
    row=1, col=2)

fig.add_trace(go.Bar(
    x = arbitrum_unique_lps_series_df['date_formatted'].values,
    y = arbitrum_unique_lps_series_df['From'].values,
    name='unique LPs'),
    row=2, col=1)

fig.add_trace(go.Scatter(
    x = arbitrum_lp_values_per_day_df['date_formatted'].values,
    y = arbitrum_lp_values_per_day_df[('usd_value', 'min')].values,
    name='min'),
    row=2, col=2)

fig.add_trace(go.Scatter(
    x = arbitrum_lp_values_per_day_df['date_formatted'].values,
    y = arbitrum_lp_values_per_day_df[('usd_value', 'max')].values,
    name='max'),
    row=2, col=2)

fig.add_trace(go.Scatter(
    x = arbitrum_lp_values_per_day_df['date_formatted'].values,
    y = arbitrum_lp_values_per_day_df[('usd_value', 'mean')].values,
    name='mean'),
    row=2, col=2)

fig.add_trace(go.Scatter(
    x = arbitrum_lp_values_per_day_df['date_formatted'].values,
    y = arbitrum_lp_values_per_day_df[('usd_value', 'median')].values,
    name='median'),
    row=2, col=2)

# # Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)

fig.update_yaxes(title_text="Cumulative Unique LPs", row=1, col=2)
fig.update_yaxes(title_text="Num Unique LPs", row=2, col=1)
fig.update_yaxes(title_text="Liquidity Value $", row=2, col=2)

fig.update_layout(height=800, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

## Arbitrum Volatility Token Liquidity Providers Detailed View

In [683]:
fig = make_subplots(
    rows=4, cols=2,
    specs=[[{"type": "indicator"}, {"type": "indicator"}],
           [{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}],
          [{"type": "scatter"}, {"type": "scatter"}]],
    subplot_titles=("BTCV-USDC Unique LPs",
                    "ETHV-USDC Unique LPs",
                    "BTCV-USDC Cumulative Unique LPs",
                    "ETHV-USDC Cumulative Unique LPs",
                    "BTCV-USDC Unique LPs Per Day",
                    "ETHV-USDC Unique LPs Per Day",
                    "BTCV-USDC Min/Max/Avg/Median Liquidity Added Values Per Day",
                    "ETHV-USDC Min/Max/Avg/Median Liquidity Added Per Day")
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = btcv_usdc_unique_lps),
row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    value = ethv_usdc_unique_lps),
row=1, col=2)

fig.add_trace(go.Bar(
    x = btcv_usdc_lps_min_date_series_df['date_formatted'],
    y = btcv_usdc_lps_min_date_series_df['cumulative_lps'].values,
    name='btcv_usdc_cumulative_unique_lps'),
    row=2, col=1)

fig.add_trace(go.Bar(
    x = ethv_usdc_lps_min_date_series_df['date_formatted'],
    y = ethv_usdc_lps_min_date_series_df['cumulative_lps'].values,
    name='ethv_usdc_cumulative_unique_lps'),
    row=2, col=2)

fig.add_trace(go.Bar(
    x = btcv_usdc_unique_lps_series_df['date_formatted'].values,
    y = btcv_usdc_unique_lps_series_df['From'].values,
    name='btcv_usdc_unique_lps'),
    row=3, col=1)

fig.add_trace(go.Bar(
    x = ethv_usdc_unique_lps_series_df['date_formatted'].values,
    y = ethv_usdc_unique_lps_series_df['From'].values,
    name='ethv_usdc_unique_lps'),
    row=3, col=2)

fig.add_trace(go.Scatter(
    x = btcv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = btcv_usdc_lp_values_per_day_df[('Value', 'min')].values,
    name='btcv_usdc_liquidity_added_value_min'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = btcv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = btcv_usdc_lp_values_per_day_df[('Value', 'max')].values,
    name='btcv_usdc_liquidity_added_value_max'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = btcv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = btcv_usdc_lp_values_per_day_df[('Value', 'mean')].values,
    name='btcv_usdc_liquidity_added_value_mean'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = btcv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = btcv_usdc_lp_values_per_day_df[('Value', 'median')].values,
    name='btcv_usdc_liquidity_added_value_median'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = ethv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = ethv_usdc_lp_values_per_day_df[('Value', 'min')].values,
    name='ethv_usdc_liquidity_added_value_min'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = ethv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = ethv_usdc_lp_values_per_day_df[('Value', 'max')].values,
    name='ethv_usdc_liquidity_added_value_max'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = ethv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = ethv_usdc_lp_values_per_day_df[('Value', 'mean')].values,
    name='ethv_usdc_liquidity_added_value_mean'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = ethv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = ethv_usdc_lp_values_per_day_df[('Value', 'median')].values,
    name='ethv_usdc_liquidity_added_value_median'),
    row=4, col=2)

# # Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=2)
fig.update_xaxes(title_text="Date", row=4, col=1)
fig.update_xaxes(title_text="Date", row=4, col=2)


fig.update_yaxes(title_text="Cumulative Unique LPs", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Unique LPs", row=2, col=2)

fig.update_yaxes(title_text="Num Unique LPs", row=3, col=1)
fig.update_yaxes(title_text="Num Unique LPs", row=3, col=2)

fig.update_yaxes(title_text="Liquidity Added Value $", row=4, col=1)
fig.update_yaxes(title_text="Liquidity Added Value $", row=4, col=2)

fig.update_layout(height=1000, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

In [684]:
fig = make_subplots(
    rows=4, cols=2,
    specs=[[{"type": "indicator"}, {"type": "indicator"}],
           [{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}],
          [{"type": "scatter"}, {"type": "scatter"}]],
    subplot_titles=("iBTCV-USDC Unique LPs",
                    "iETHV-USDC Unique LPs",
                    "iBTCV-USDC Cumulative Unique LPs",
                    "iETHV-USDC Cumulative Unique LPs",
                    "iBTCV-USDC Unique LPs Per Day",
                    "iETHV-USDC Unique LPs Per Day",
                    "iBTCV-USDC Min/Max/Avg/Median Liquidity Added Values Per Day",
                    "iETHV-USDC Min/Max/Avg/Median Liquidity Added Per Day")
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = ibtcv_usdc_unique_lps),
row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    value = iethv_usdc_unique_lps),
row=1, col=2)

fig.add_trace(go.Bar(
    x = ibtcv_usdc_lps_min_date_series_df['date_formatted'],
    y = ibtcv_usdc_lps_min_date_series_df['cumulative_lps'].values,
    name='ibtcv_usdc_cumulative_unique_lps'),
    row=2, col=1)

fig.add_trace(go.Bar(
    x = iethv_usdc_lps_min_date_series_df['date_formatted'],
    y = iethv_usdc_lps_min_date_series_df['cumulative_lps'].values,
    name='iethv_usdc_cumulative_unique_lps'),
    row=2, col=2)

fig.add_trace(go.Bar(
    x = ibtcv_usdc_unique_lps_series_df['date_formatted'].values,
    y = ibtcv_usdc_unique_lps_series_df['From'].values,
    name='ibtcv_usdc_unique_lps'),
    row=3, col=1)

fig.add_trace(go.Bar(
    x = iethv_usdc_unique_lps_series_df['date_formatted'].values,
    y = iethv_usdc_unique_lps_series_df['From'].values,
    name='iethv_usdc_unique_lps'),
    row=3, col=2)

fig.add_trace(go.Scatter(
    x = ibtcv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = ibtcv_usdc_lp_values_per_day_df[('Value', 'min')].values,
    name='ibtcv_usdc_liquidity_added_value_min'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = ibtcv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = ibtcv_usdc_lp_values_per_day_df[('Value', 'max')].values,
    name='ibtcv_usdc_liquidity_added_value_max'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = ibtcv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = ibtcv_usdc_lp_values_per_day_df[('Value', 'mean')].values,
    name='ibtcv_usdc_liquidity_added_value_mean'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = ibtcv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = ibtcv_usdc_lp_values_per_day_df[('Value', 'median')].values,
    name='ibtcv_usdc_liquidity_added_value_median'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x = iethv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = iethv_usdc_lp_values_per_day_df[('Value', 'min')].values,
    name='iethv_usdc_liquidity_added_value_min'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = iethv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = iethv_usdc_lp_values_per_day_df[('Value', 'max')].values,
    name='iethv_usdc_liquidity_added_value_max'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = iethv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = iethv_usdc_lp_values_per_day_df[('Value', 'mean')].values,
    name='iethv_usdc_liquidity_added_value_mean'),
    row=4, col=2)

fig.add_trace(go.Scatter(
    x = iethv_usdc_lp_values_per_day_df['date_formatted'].values,
    y = iethv_usdc_lp_values_per_day_df[('Value', 'median')].values,
    name='iethv_usdc_liquidity_added_value_median'),
    row=4, col=2)

# # Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=2)
fig.update_xaxes(title_text="Date", row=4, col=1)
fig.update_xaxes(title_text="Date", row=4, col=2)


fig.update_yaxes(title_text="Cumulative Unique LPs", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Unique LPs", row=2, col=2)

fig.update_yaxes(title_text="Num Unique LPs", row=3, col=1)
fig.update_yaxes(title_text="Num Unique LPs", row=3, col=2)

fig.update_yaxes(title_text="Liquidity Added Value $", row=4, col=1)
fig.update_yaxes(title_text="Liquidity Added Value $", row=4, col=2)

fig.update_layout(height=1000, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

## Arbitrum Volatility Token Traders

In [685]:
# arbitrum_lps_traders_df['tx_type_label'].unique()
arbitrum_traders_df = arbitrum_lps_traders_df[arbitrum_lps_traders_df['tx_type_label'] == 'Swap']

def exchange_pool(row):
    if row['From'] == '0x3396e2cb9e4f1eb4fc06028ab07a8db0ef63b486' or row['To'] == '0x3396e2cb9e4f1eb4fc06028ab07a8db0ef63b486':
        return 'BTCV-USDC'
    elif row['From'] == '0x8278591bbbbe7d86e783507e7f6320ffc9c021e5' or row['To'] == '0x8278591bbbbe7d86e783507e7f6320ffc9c021e5':
        return 'ETHV-USDC'
    elif row['From'] == '0xc80956377cea73f64aaa3613d6692eabdf4fd4c8' or row['To'] == '0xc80956377cea73f64aaa3613d6692eabdf4fd4c8':
        return 'iBTCV-USDC'
    elif row['From']  == '0x5497ca4be41d2ae0222cc4c41c43d189ed894ffd' or row['To'] == '0x5497ca4be41d2ae0222cc4c41c43d189ed894ffd':
        return 'iETHV-USDC'
    return 'UNKNOWN'

arbitrum_traders_df["exchange_pool"] = arbitrum_traders_df.apply(exchange_pool, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [686]:
# Arbitrum Traders - Filter for the IN side of the trade as this will show the user address
arbitrum_traders_df = arbitrum_traders_df[arbitrum_traders_df['Type'] == 'IN']

btcv_usdc_traders_df = arbitrum_traders_df[(arbitrum_traders_df['exchange_pool'] == 'BTCV-USDC') &
                                       (arbitrum_traders_df['Type'] == 'IN')]
ethv_usdc_traders_df = arbitrum_traders_df[(arbitrum_traders_df['exchange_pool'] == 'ETHV-USDC') &
                                       (arbitrum_traders_df['Type'] == 'IN')]
ibtcv_usdc_traders_df = arbitrum_traders_df[(arbitrum_traders_df['exchange_pool'] == 'iBTCV-USDC') &
                                       (arbitrum_traders_df['Type'] == 'IN')]
iethv_usdc_traders_df = arbitrum_traders_df[(arbitrum_traders_df['exchange_pool'] == 'iETHV-USDC') &
                                       (arbitrum_traders_df['Type'] == 'IN')]

# Calculate number of unique arbitrum traders
arbitrum_unique_traders = arbitrum_traders_df['From'].nunique()
btcv_usdc_unique_traders = btcv_usdc_traders_df['From'].nunique()
ethv_usdc_unique_traders = ethv_usdc_traders_df['From'].nunique()
ibtcv_usdc_unique_traders = ibtcv_usdc_traders_df['From'].nunique()
iethv_usdc_unique_traders = iethv_usdc_traders_df['From'].nunique()

In [687]:
# Create traders date series sequence for cumulative calculations etc
min_date = arbitrum_traders_df['date_formatted'].min()
max_date = arbitrum_traders_df['date_formatted'].max()
arbitrum_traders_date_series_df = pd.DataFrame({'date_formatted': pd.date_range(start=min_date, end=max_date).values})
arbitrum_traders_date_series_df['date_formatted'] = pd.to_datetime(arbitrum_traders_date_series_df['date_formatted']).dt.date

In [688]:
arbitrum_traders_min_date_df = arbitrum_traders_df.groupby('From')['date_formatted'].agg('min').reset_index()
arbitrum_traders_min_date_df = arbitrum_traders_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
arbitrum_traders_min_date_series_df = arbitrum_traders_date_series_df.merge(arbitrum_traders_min_date_df, on='date_formatted', how='left').fillna(0)
arbitrum_traders_min_date_series_df['cumulative_traders'] = arbitrum_traders_min_date_series_df['From'].cumsum(axis=0)

btcv_usdc_traders_min_date_df = btcv_usdc_traders_df.groupby('From')['date_formatted'].agg('min').reset_index()
btcv_usdc_traders_min_date_df = btcv_usdc_traders_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
btcv_usdc_traders_min_date_series_df = arbitrum_traders_date_series_df.merge(btcv_usdc_traders_min_date_df, on='date_formatted', how='left').fillna(0)
btcv_usdc_traders_min_date_series_df['cumulative_traders'] = btcv_usdc_traders_min_date_series_df['From'].cumsum(axis=0)

ethv_usdc_traders_min_date_df = ethv_usdc_traders_df.groupby('From')['date_formatted'].agg('min').reset_index()
ethv_usdc_traders_min_date_df = ethv_usdc_traders_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
ethv_usdc_traders_min_date_series_df = arbitrum_traders_date_series_df.merge(ethv_usdc_traders_min_date_df, on='date_formatted', how='left').fillna(0)
ethv_usdc_traders_min_date_series_df['cumulative_traders'] = ethv_usdc_traders_min_date_series_df['From'].cumsum(axis=0)

ibtcv_usdc_traders_min_date_df = ibtcv_usdc_traders_df.groupby('From')['date_formatted'].agg('min').reset_index()
ibtcv_usdc_traders_min_date_df = ibtcv_usdc_traders_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
ibtcv_usdc_traders_min_date_series_df = arbitrum_traders_date_series_df.merge(ibtcv_usdc_traders_min_date_df, on='date_formatted', how='left').fillna(0)
ibtcv_usdc_traders_min_date_series_df['cumulative_traders'] = ibtcv_usdc_traders_min_date_series_df['From'].cumsum(axis=0)

iethv_usdc_traders_min_date_df = iethv_usdc_traders_df.groupby('From')['date_formatted'].agg('min').reset_index()
iethv_usdc_traders_min_date_df = iethv_usdc_traders_min_date_df.groupby('date_formatted').agg({'From': pd.Series.nunique}).sort_values(['date_formatted'])
iethv_usdc_traders_min_date_series_df = arbitrum_traders_date_series_df.merge(iethv_usdc_traders_min_date_df, on='date_formatted', how='left').fillna(0)
iethv_usdc_traders_min_date_series_df['cumulative_traders'] = iethv_usdc_traders_min_date_series_df['From'].cumsum(axis=0)

In [689]:
# Calculate unique traders per day over time
arbitrum_unique_traders_per_day_df = arbitrum_traders_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
arbitrum_unique_traders_series_df = arbitrum_traders_date_series_df.merge(arbitrum_unique_traders_per_day_df, on='date_formatted', how='left').fillna(0)

btcv_usdc_unique_traders_per_day_df = btcv_usdc_traders_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
btcv_usdc_unique_traders_series_df = arbitrum_traders_date_series_df.merge(btcv_usdc_unique_traders_per_day_df, on='date_formatted', how='left').fillna(0)

ethv_usdc_unique_traders_per_day_df = ethv_usdc_traders_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
ethv_usdc_unique_traders_series_df = arbitrum_traders_date_series_df.merge(ethv_usdc_unique_traders_per_day_df, on='date_formatted', how='left').fillna(0)

ibtcv_usdc_unique_traders_per_day_df = ibtcv_usdc_traders_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
ibtcv_usdc_unique_traders_series_df = arbitrum_traders_date_series_df.merge(ibtcv_usdc_unique_traders_per_day_df, on='date_formatted', how='left').fillna(0)

iethv_usdc_unique_traders_per_day_df = iethv_usdc_traders_df.groupby('date_formatted').agg({'From': pd.Series.nunique})
iethv_usdc_unique_traders_series_df = arbitrum_traders_date_series_df.merge(iethv_usdc_unique_traders_per_day_df, on='date_formatted', how='left').fillna(0)

In [690]:
# Reset arbitrum traders as all required for the trading volume to filter for the USDC components
arbitrum_all_traders_df = arbitrum_lps_traders_df[arbitrum_lps_traders_df['tx_type_label'] == 'Swap']
arbitrum_all_traders_df["exchange_pool"] = arbitrum_all_traders_df.apply(exchange_pool, axis=1)

# Total and Pool Trade Volumes USDC side
arbitrum_trade_vol_df = arbitrum_all_traders_df[arbitrum_all_traders_df['TokenSymbol'] == 'USDC']

btcv_usdc_trade_vol_df = arbitrum_all_traders_df[(arbitrum_all_traders_df['exchange_pool'] == 'BTCV-USDC') &
                                       (arbitrum_all_traders_df['TokenSymbol'] == 'USDC')]
ethv_usdc_trade_vol_df = arbitrum_all_traders_df[(arbitrum_all_traders_df['exchange_pool'] == 'ETHV-USDC') &
                                       (arbitrum_all_traders_df['TokenSymbol'] == 'USDC')]
ibtcv_usdc_trade_vol_df = arbitrum_all_traders_df[(arbitrum_all_traders_df['exchange_pool'] == 'iBTCV-USDC') &
                                       (arbitrum_all_traders_df['TokenSymbol'] == 'USDC')]
iethv_usdc_trade_vol_df = arbitrum_all_traders_df[(arbitrum_all_traders_df['exchange_pool'] == 'iETHV-USDC') &
                                       (arbitrum_all_traders_df['TokenSymbol'] == 'USDC')]

# Calculate cumulative trading volume for total + each swap pool
arbitrum_cumulative_trade_vol = arbitrum_trade_vol_df['Value'].sum()
btcv_usdc_cumulative_trade_vol = btcv_usdc_trade_vol_df['Value'].sum()
ethv_usdc_cumulative_trade_vol = ethv_usdc_trade_vol_df['Value'].sum()
ibtcv_usdc_cumulative_trade_vol = ibtcv_usdc_trade_vol_df['Value'].sum()
iethv_usdc_cumulative_trade_vol = iethv_usdc_trade_vol_df['Value'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [691]:
# Cumulative Trading Volume (USD) over time
arbitrum_trade_vol_df = arbitrum_trade_vol_df.groupby('date_formatted').agg({'Value': 'sum'}).sort_values(['date_formatted']).reset_index()
arbitrum_trade_vol_date_series_df = arbitrum_traders_date_series_df.merge(arbitrum_trade_vol_df, on='date_formatted', how='left').fillna(0)
arbitrum_trade_vol_date_series_df['cumulative_trading_volume'] = arbitrum_trade_vol_date_series_df['Value'].cumsum(axis=0)

btcv_usdc_trade_vol_df = btcv_usdc_trade_vol_df.groupby('date_formatted').agg({'Value': 'sum'}).sort_values(['date_formatted']).reset_index()
btcv_usdc_trade_vol_date_series_df = arbitrum_traders_date_series_df.merge(btcv_usdc_trade_vol_df, on='date_formatted', how='left').fillna(0)
btcv_usdc_trade_vol_date_series_df['cumulative_trading_volume'] = btcv_usdc_trade_vol_date_series_df['Value'].cumsum(axis=0)

ethv_usdc_trade_vol_df = ethv_usdc_trade_vol_df.groupby('date_formatted').agg({'Value': 'sum'}).sort_values(['date_formatted']).reset_index()
ethv_usdc_trade_vol_date_series_df = arbitrum_traders_date_series_df.merge(ethv_usdc_trade_vol_df, on='date_formatted', how='left').fillna(0)
ethv_usdc_trade_vol_date_series_df['cumulative_trading_volume'] = ethv_usdc_trade_vol_date_series_df['Value'].cumsum(axis=0)

ibtcv_usdc_trade_vol_df = ibtcv_usdc_trade_vol_df.groupby('date_formatted').agg({'Value': 'sum'}).sort_values(['date_formatted']).reset_index()
ibtcv_usdc_trade_vol_date_series_df = arbitrum_traders_date_series_df.merge(ibtcv_usdc_trade_vol_df, on='date_formatted', how='left').fillna(0)
ibtcv_usdc_trade_vol_date_series_df['cumulative_trading_volume'] = ibtcv_usdc_trade_vol_date_series_df['Value'].cumsum(axis=0)

iethv_usdc_trade_vol_df = iethv_usdc_trade_vol_df.groupby('date_formatted').agg({'Value': 'sum'}).sort_values(['date_formatted']).reset_index()
iethv_usdc_trade_vol_date_series_df = arbitrum_traders_date_series_df.merge(iethv_usdc_trade_vol_df, on='date_formatted', how='left').fillna(0)
iethv_usdc_trade_vol_date_series_df['cumulative_trading_volume'] = iethv_usdc_trade_vol_date_series_df['Value'].cumsum(axis=0)

In [692]:
fig = make_subplots(
    rows=3, cols=2,
    specs=[[{"type": "indicator"}, {"type": "bar"}],
          [{"type": "bar"}, {"type": "indicator"}],
           [{"type": "bar"}, {"type": "indicator"}]],
    subplot_titles=("Arbitrum Unique Traders",
                    "Arbitrum Cumulative Unique Traders", 
                    "Arbitrum Unique Traders Per Day",
                    "Arbitrum Trading Volume USD",
                    "Arbitrum Cumulative Trading Volume USD",
                    "")
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = arbitrum_unique_traders),
row=1, col=1)

fig.add_trace(go.Bar(
    x = arbitrum_traders_min_date_series_df['date_formatted'],
    y = arbitrum_traders_min_date_series_df['cumulative_traders'].values,
    name='cumulative unique traders'),
    row=1, col=2)

fig.add_trace(go.Bar(
    x = arbitrum_unique_traders_series_df['date_formatted'].values,
    y = arbitrum_unique_traders_series_df['From'].values,
    name='unique traders'),
    row=2, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    number = {'prefix': "$"},
    value = arbitrum_cumulative_trade_vol),
    row=2, col=2)

fig.add_trace(go.Bar(
    x = arbitrum_trade_vol_date_series_df['date_formatted'].values,
    y = arbitrum_trade_vol_date_series_df['cumulative_trading_volume'].values,
    name='unique traders'),
    row=3, col=1)

# # Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=3, col=1)

fig.update_yaxes(title_text="Cumulative Unique Traders", row=1, col=2)
fig.update_yaxes(title_text="Num Unique Traders", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Trading Volume $", row=3, col=1)

fig.update_layout(height=800, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

## Arbitrum Volatility Token Traders Detailed View

In [693]:
fig = make_subplots(
    rows=5, cols=2,
    specs=[[{"type": "indicator"}, {"type": "indicator"}],
          [{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}],
           [{"type": "indicator"}, {"type": "indicator"}],
           [{"type": "bar"}, {"type": "bar"}],],
    subplot_titles=("BTCV-USDC Unique Traders",
                    "ETHV-USDC Unique Traders",
                    "BTCV-USDC Cumulative Unique Traders",
                    "ETHV-USDC Cumulative Unique Traders",
                    "BTCV-USDC Unique Traders Per Day",
                    "ETHV-USDC Unique Traders Per Day",
                    "BTCV-USDC Trading Volume USD",
                    "ETHV-USDC Trading Volume USD",
                    "BTCV-USDC Cumulative Trading Volume USD",
                    "ETHV-USDC Cumulative Trading Volume USD")
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = btcv_usdc_unique_traders),
row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    value = ethv_usdc_unique_traders),
row=1, col=2)

fig.add_trace(go.Bar(
    x = btcv_usdc_traders_min_date_series_df['date_formatted'],
    y = btcv_usdc_traders_min_date_series_df['cumulative_traders'].values,
    name='btcv_usdc_cumulative unique traders'),
    row=2, col=1)

fig.add_trace(go.Bar(
    x = ethv_usdc_traders_min_date_series_df['date_formatted'].values,
    y = ethv_usdc_traders_min_date_series_df['cumulative_traders'].values,
    name='ethv_usdc_cumulative unique traders'),
    row=2, col=2)

fig.add_trace(go.Bar(
    x = btcv_usdc_unique_traders_series_df['date_formatted'],
    y = btcv_usdc_unique_traders_series_df['From'].values,
    name='btcv_usdc_unique traders'),
    row=3, col=1)

fig.add_trace(go.Bar(
    x = ethv_usdc_unique_traders_series_df['date_formatted'].values,
    y = ethv_usdc_unique_traders_series_df['From'].values,
    name='ethv_usdc_unique traders'),
    row=3, col=2)

fig.add_trace(go.Indicator(
    mode = "number",
    number = {'prefix': "$"},
    value = btcv_usdc_cumulative_trade_vol),
    row=4, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    number = {'prefix': "$"},
    value = ethv_usdc_cumulative_trade_vol),
    row=4, col=2)

fig.add_trace(go.Bar(
    x = btcv_usdc_trade_vol_date_series_df['date_formatted'].values,
    y = btcv_usdc_trade_vol_date_series_df['cumulative_trading_volume'].values,
    name='btcv_usdc_trading_volume'),
    row=5, col=1)

fig.add_trace(go.Bar(
    x = ethv_usdc_trade_vol_date_series_df['date_formatted'].values,
    y = ethv_usdc_trade_vol_date_series_df['cumulative_trading_volume'].values,
    name='ethv_usdc_trading_volume'),
    row=5, col=2)

# # Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=2)
fig.update_xaxes(title_text="Date", row=5, col=1)
fig.update_xaxes(title_text="Date", row=5, col=2)

fig.update_yaxes(title_text="Cumulative Unique Traders", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Unique Traders", row=2, col=2)
fig.update_yaxes(title_text="Num Unique Traders", row=3, col=1)
fig.update_yaxes(title_text="Num Unique Traders", row=3, col=2)
fig.update_yaxes(title_text="Cumulative Trading Volume $", row=5, col=1)
fig.update_yaxes(title_text="Cumulative Trading Volume $", row=5, col=2)

fig.update_layout(height=1200, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()

In [694]:
fig = make_subplots(
    rows=5, cols=2,
    specs=[[{"type": "indicator"}, {"type": "indicator"}],
          [{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}],
           [{"type": "indicator"}, {"type": "indicator"}],
           [{"type": "bar"}, {"type": "bar"}],],
    subplot_titles=("iBTCV-USDC Unique Traders",
                    "iETHV-USDC Unique Traders",
                    "iBTCV-USDC Cumulative Unique Traders",
                    "iETHV-USDC Cumulative Unique Traders",
                    "iBTCV-USDC Unique Traders Per Day",
                    "iETHV-USDC Unique Traders Per Day",
                    "iBTCV-USDC Trading Volume USD",
                    "iETHV-USDC Trading Volume USD",
                    "iBTCV-USDC Cumulative Trading Volume USD",
                    "iETHV-USDC Cumulative Trading Volume USD")
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = ibtcv_usdc_unique_traders),
row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    value = iethv_usdc_unique_traders),
row=1, col=2)

fig.add_trace(go.Bar(
    x = ibtcv_usdc_traders_min_date_series_df['date_formatted'],
    y = ibtcv_usdc_traders_min_date_series_df['cumulative_traders'].values,
    name='ibtcv_usdc_cumulative unique traders'),
    row=2, col=1)

fig.add_trace(go.Bar(
    x = iethv_usdc_traders_min_date_series_df['date_formatted'].values,
    y = iethv_usdc_traders_min_date_series_df['cumulative_traders'].values,
    name='iethv_usdc_cumulative unique traders'),
    row=2, col=2)

fig.add_trace(go.Bar(
    x = ibtcv_usdc_unique_traders_series_df['date_formatted'],
    y = ibtcv_usdc_unique_traders_series_df['From'].values,
    name='ibtcv_usdc_unique traders'),
    row=3, col=1)

fig.add_trace(go.Bar(
    x = iethv_usdc_unique_traders_series_df['date_formatted'].values,
    y = iethv_usdc_unique_traders_series_df['From'].values,
    name='iethv_usdc_unique traders'),
    row=3, col=2)

fig.add_trace(go.Indicator(
    mode = "number",
    number = {'prefix': "$"},
    value = ibtcv_usdc_cumulative_trade_vol),
    row=4, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    number = {'prefix': "$"},
    value = iethv_usdc_cumulative_trade_vol),
    row=4, col=2)

fig.add_trace(go.Bar(
    x = ibtcv_usdc_trade_vol_date_series_df['date_formatted'].values,
    y = ibtcv_usdc_trade_vol_date_series_df['cumulative_trading_volume'].values,
    name='ibtcv_usdc_trading_volume'),
    row=5, col=1)

fig.add_trace(go.Bar(
    x = iethv_usdc_trade_vol_date_series_df['date_formatted'].values,
    y = iethv_usdc_trade_vol_date_series_df['cumulative_trading_volume'].values,
    name='iethv_usdc_trading_volume'),
    row=5, col=2)

# # Row 2 axis labelling
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=2)
fig.update_xaxes(title_text="Date", row=5, col=1)
fig.update_xaxes(title_text="Date", row=5, col=2)

fig.update_yaxes(title_text="Cumulative Unique Traders", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Unique Traders", row=2, col=2)
fig.update_yaxes(title_text="Num Unique Traders", row=3, col=1)
fig.update_yaxes(title_text="Num Unique Traders", row=3, col=2)
fig.update_yaxes(title_text="Cumulative Trading Volume $", row=5, col=1)
fig.update_yaxes(title_text="Cumulative Trading Volume $", row=5, col=2)

fig.update_layout(height=1200, 
                  showlegend=True,
                  title_font_size=24,
                  title_xanchor='left')

fig.show()